In [ ]:
 %%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

EMT_DP_SP_VS_RLC

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import re

#%matplotlib widget

epsilon = 1e-12

## Voltage Source + Resistor

In [ ]:
model_name = 'VoltageSource_Resistor'

path_SP = 'logs/' + 'SP_' + model_name + '/'
dpsim_result_file_SP = path_SP  + 'SP_' + model_name + '.csv'
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = 'logs/' + 'DP_' + model_name + '/'
dpsim_result_file_DP = path_DP  + 'DP_' + model_name + '.csv'
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = 'logs/' + 'EMT_' + model_name + '/'
dpsim_result_file_EMT = path_EMT  + 'EMT_' + model_name + '.csv'
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ['v1_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['v1_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name+' (DP)', linestyle='-.')
    
# SP
var_names = ['v1_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_SP_shifted[var_name].time, ts_dpsim_SP_shifted[var_name].values, label=var_name+' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ['iR_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['iR_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name+' (DP)', linestyle='-.')
    
# SP
var_names = ['iR_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_SP_shifted[var_name].time, ts_dpsim_SP_shifted[var_name].values, label=var_name+' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [('v1_shift', 'v1_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_shift', 'iR_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

### Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_shift', 'v1_shift'), ('iR_shift', 'iR_shift')]:
    compare_errors_abs.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < epsilon

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [('v1_0', 'v1_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_0', 'iR_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_0', 'v1_shift'), ('iR_0', 'iR_shift')]:
    compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 1e-3

## Voltage Source + Resistor + Inductor

In [ ]:
model_name = 'VoltageSource_Inductor'

path_SP = 'logs/' + 'SP_' + model_name + '/'
dpsim_result_file_SP = path_SP  + 'SP_' + model_name + '.csv'
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = 'logs/' + 'DP_' + model_name + '/'
dpsim_result_file_DP = path_DP  + 'DP_' + model_name + '.csv'
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = 'logs/' + 'EMT_' + model_name + '/'
dpsim_result_file_EMT = path_EMT  + 'EMT_' + model_name + '.csv'
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ['v1_0', 'v2_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['v1_shift', 'v2_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name+' (DP)', linestyle='-.')
    
# SP
var_names = ['v1_shift', 'v2_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_SP_shifted[var_name].time, ts_dpsim_SP_shifted[var_name].values, label=var_name+' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ['iR_0', 'iL_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['iR_shift', 'iL_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name+' (DP)', linestyle=':')
    
# SP
var_names = ['iR_shift', 'iL_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_SP_shifted[var_name].time, ts_dpsim_SP_shifted[var_name].values, label=var_name+' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [('v1_shift', 'v1_shift'), ('v2_shift', 'v2_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_shift', 'iR_shift'), ('iL_shift', 'iL_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

## Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_shift', 'v1_shift'), ('v2_shift', 'v2_shift'), ('iR_shift', 'iR_shift'), ('iL_shift', 'iL_shift')]:
    compare_errors_abs.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 3e-1

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [('v1_0', 'v1_shift'), ('v2_0', 'v2_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_0', 'iR_shift'), ('iL_0', 'iL_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_0', 'v1_shift'), ('v2_0', 'v2_shift'), ('iR_0', 'iR_shift'), ('iL_0', 'iL_shift')]:
    compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 1e-3

## Voltage Source + Resistor + Capacitor

In [ ]:
model_name = 'VoltageSource_Capacitor'

path_SP = 'logs/' + 'SP_' + model_name + '/'
dpsim_result_file_SP = path_SP  + 'SP_' + model_name + '.csv'
ts_dpsim_SP = read_timeseries_csv(dpsim_result_file_SP)
ts_dpsim_SP_shifted = ts.frequency_shift_list(ts_dpsim_SP, 50)

path_DP = 'logs/' + 'DP_' + model_name + '/'
dpsim_result_file_DP = path_DP  + 'DP_' + model_name + '.csv'
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)
ts_dpsim_DP_shifted = ts.frequency_shift_list(ts_dpsim_DP, 50)

path_EMT = 'logs/' + 'EMT_' + model_name + '/'
dpsim_result_file_EMT = path_EMT  + 'EMT_' + model_name + '.csv'
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

### Plot voltages

In [ ]:
plt.figure()

# EMT
var_names = ['v1_0', 'v2_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['v1_shift', 'v2_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name + ' (DP)', linestyle='-.')
    
# SP
var_names = ['v1_shift', 'v2_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_SP_shifted[var_name].time, ts_dpsim_SP_shifted[var_name].values, label=var_name + ' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Plot currents

In [ ]:
plt.figure()

# EMT
var_names = ['iR_0', 'iC_0']
for var_name in var_names: 
    plt.plot(ts_dpsim_EMT[var_name].time, np.sqrt(3/2)*ts_dpsim_EMT[var_name].values, label=var_name)
    
# DP
var_names = ['iR_shift', 'iC_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name + ' (DP)', linestyle=':')
    
# SP
var_names = ['iR_shift', 'iC_shift']
for var_name in var_names: 
    plt.plot(ts_dpsim_DP_shifted[var_name].time, ts_dpsim_DP_shifted[var_name].values, label=var_name + ' (SP)', linestyle=':')
    
plt.legend()
plt.show()

### Comparison SP vs. DP

In [ ]:
plt.figure()
for name in [('v1_shift', 'v1_shift'), ('v2_shift', 'v2_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_shift', 'iR_shift'), ('iC_shift', 'iC_shift')]:
    plt.plot(ts_dpsim_SP_shifted[name[0]].time, ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' (SP) vs. '+name[1]+' (DP)')
plt.legend()

### Assertion SP vs. DP

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_shift', 'v1_shift'), ('v2_shift', 'v2_shift'), ('iR_shift', 'iR_shift'), ('iC_shift', 'iC_shift')]:
    compare_errors_abs.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(ts_dpsim_SP_shifted[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 6e-1

### Comparison DP vs. EMT

In [ ]:
plt.figure()
for name in [('v1_0', 'v1_shift'), ('v2_0', 'v2_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

In [ ]:
plt.figure()
for name in [('iR_0', 'iR_shift'), ('iC_0', 'iC_shift')]:
    plt.plot(ts_dpsim_EMT[name[0]].time, np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values, label=name[0]+' vs. '+name[1])
plt.legend()

### Assertion DP vs. EMT

In [ ]:
compare_errors_abs = []
compare_errors_rel = []
for name in [('v1_0', 'v1_shift'), ('v2_0', 'v2_shift'), ('iR_0', 'iR_shift'), ('iC_0', 'iC_shift')]:
    compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max())
    compare_errors_rel.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values - ts_dpsim_DP_shifted[name[1]].values).max()/ts_dpsim_DP_shifted[name[1]].values.max())
    print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    print(name[0]+' vs. '+name[1] + ' (rel): ' + str(compare_errors_rel[-1]))
print('Max rel error: '+ '{:.2}'.format(np.max(compare_errors_rel)*100) +'%')
assert np.max(compare_errors_rel) < 1e-3